In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse, parse_qs, urlencode
import time
import re

def get_page_content(url, retries=3):
    """지정된 URL의 HTML 내용을 가져옵니다."""
    for attempt in range(retries):
        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
            response.raise_for_status()
            time.sleep(1)  # 서버 부하 방지를 위한 1초 딜레이
            return response.text
        except requests.RequestException as e:
            print(f"페이지 가져오기 오류 {url} (시도 {attempt+1}/{retries}): {e}")
            if attempt + 1 == retries:
                return None
            time.sleep(2)  # 재시도 전 2초 대기
    return None

def parse_notice_content(url):
    """공지사항 상세 페이지에서 본문 내용을 추출합니다."""
    print(f"상세 페이지 크롤링: {url}")
    html_content = get_page_content(url)
    if not html_content:
        print(f"상세 페이지 가져오기 실패: {url}")
        return ""

    soup = BeautifulSoup(html_content, 'html.parser')
    content_div = soup.find('div', id='bbs_ntt_cn_con')
    if not content_div:
        print(f"본문 내용을 찾을 수 없습니다: {url}")
        return ""

    content = ' '.join(content_div.get_text(strip=True).split())
    return content if content else ""

def parse_notices(html_content, base_url):
    """HTML에서 공지사항을 추출하고 상세 페이지 내용을 가져옵니다."""
    soup = BeautifulSoup(html_content, 'html.parser')
    notices = []

    notice_table = soup.find('table', class_='bbs_default list')
    if not notice_table:
        print("공지사항 테이블을 찾을 수 없습니다.")
        return notices, False

    tbody = notice_table.find('tbody', class_='tb')
    if not tbody:
        print("테이블 본문을 찾을 수 없습니다.")
        return notices, False

    notice_rows = tbody.find_all('tr')
    if not notice_rows:
        print("공지사항 행을 찾을 수 없습니다.")
        return notices, False

    for row in notice_rows:
        columns = row.find_all('td')
        if len(columns) < 7:
            continue

        number = columns[0].get_text(strip=True)
        if number == '[공지]':
            print(f"[공지] 항목 제외: {columns[2].get_text(strip=True)}")
            continue

        campus = columns[1].get_text(strip=True)
        if campus == '춘천':
            print(f"춘천 캠퍼스 제외: {columns[2].get_text(strip=True)}")
            continue

        notice = {}
        notice['number'] = number
        notice['campus'] = campus
        notice['title'] = columns[2].get_text(strip=True)

        title_cell = columns[2].find('a')
        ntt_no = None
        if title_cell:
            href = title_cell.get('href', '')
            match = re.search(r'nttNo=(\d+)', href)
            if match:
                ntt_no = match.group(1)
            onclick = title_cell.get('onclick', '')
            match = re.search(r'goDetail\((\d+)\)', onclick)
            if match:
                ntt_no = match.group(1)

        if ntt_no:
            notice['link'] = f"{base_url}/www/selectBbsNttView.do?bbsNo=37&nttNo={ntt_no}"
        else:
            notice['link'] = ''
            print(f"nttNo 추출 실패: 제목={notice['title']}")

        notice['author'] = columns[3].get_text(strip=True)
        notice['date'] = columns[5].get_text(strip=True)
        notice['views'] = columns[6].get_text(strip=True)
        notice['content'] = parse_notice_content(notice['link']) if notice['link'] else ""

        print(f"\n공지사항: {notice['title']}")
        print(f"URL: {notice['link']}")
        if notice['content']:
            preview = notice['content'][:200] + ('...' if len(notice['content']) > 200 else '')
            print(f"본문 내용: {preview}")
        else:
            print("본문 내용: 없음")
        print("-" * 80)

        notices.append(notice)

    next_page_exists = bool(soup.select_one('a.next'))
    return notices, next_page_exists

def get_total_pages(html_content):
    """총 페이지 수를 추출합니다."""
    soup = BeautifulSoup(html_content, 'html.parser')
    paging_div = soup.find('div', class_='paging')
    if not paging_div:
        return 475

    page_links = paging_div.find_all('a')
    page_numbers = []

    for link in page_links:
        href = link.get('href', '')
        match = re.search(r'pageIndex=(\d+)', href)
        if match:
            page_numbers.append(int(match.group(1)))
        elif 'goPage' in href:
            match = re.search(r'goPage\((\d+)\)', href)
            if match:
                page_numbers.append(int(match.group(1)))

    return max(page_numbers, default=475)

def crawl_all_notices(start_url, estimated_total_items=4748, page_size=10):
    """모든 페이지의 공지사항과 본문 내용을 크롤링합니다."""
    all_notices = []
    current_page = 1
    url_components = urlparse(start_url)
    base_url = f"{url_components.scheme}://{url_components.netloc}"
    query_params = parse_qs(url_components.query)

    max_pages = (estimated_total_items + page_size - 1) // page_size
    print(f"추정 최대 페이지 수: {max_pages}")

    while current_page <= max_pages:
        print(f"페이지 {current_page}/{max_pages} 크롤링 중...")
        query_params['pageIndex'] = [str(current_page)]
        new_query = urlencode(query_params, doseq=True)
        current_url = f"{url_components.scheme}://{url_components.netloc}{url_components.path}?{new_query}"

        html_content = get_page_content(current_url)
        if not html_content:
            print(f"페이지 {current_page} 가져오기 실패. 건너뜁니다.")
            current_page += 1
            continue

        notices, next_page_exists = parse_notices(html_content, base_url)
        if not notices:
            print(f"페이지 {current_page}에서 공지사항이 없습니다. 크롤링 종료.")
            break

        all_notices.extend(notices)
        print(f"페이지 {current_page}: {len(notices)}개 공지사항 수집 (총 {len(all_notices)}개, 다음 페이지 존재: {next_page_exists})")

        if current_page % 100 == 0:
            print(f"진행 상황: {current_page}/{max_pages} 페이지 완료")
            save_to_csv(all_notices, f'kangwon_notices_partial_{current_page}.csv')

        if current_page == max_pages:
            print(f"마지막 페이지({current_page}) 도달. 공지사항 수: {len(notices)}, 다음 페이지 존재: {next_page_exists}")

        current_page += 1

        if not next_page_exists and len(notices) < page_size:
            print("다음 페이지가 없으며, 공지사항이 적게 수집되었습니다. 크롤링 종료.")
            break

    return all_notices

def save_to_csv(notices, filename='kangwon_notices.csv'):
    """공지사항을 CSV 파일로 저장합니다."""
    if not notices:
        print("저장할 공지사항이 없습니다.")
        return

    df = pd.DataFrame(notices)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"{len(notices)}개의 공지사항을 {filename}에 저장했습니다.")

if __name__ == "__main__":
    start_url = "https://wwwk.kangwon.ac.kr/www/selectBbsNttList.do?bbsNo=37&key=1176&pageUnit=10&pageIndex=1"
    notices = crawl_all_notices(start_url)
    save_to_csv(notices)